# Face Recognition – TASK 06!
# Task Description 📄

## ❄️ Create a program that perform below mentioned task upon recognizing a particular face. 

## 📌 When it recognize your face then - 
### 👉 It send mail to your mail id by writing this is face of your_name. 
### 👉 Second it send whatsapp message to your friend, it can be anything. 

## 📌 When it recognize second  face, it can be your friend or family members face.
### 👉 Create EC2 instance in the AWS using CLI. 
### 👉 Create 5 GB EBS volume and attach it to the instance. 

## Step 1 - Create Training Data

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h+10, x:x+w+10]

    return cropped_face

# Initialize Webcam
user_id = input('ENTER USER-ID : ')
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
#         face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/User_' + str(user_id) + "_" + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        face1 = cv2.flip(face,1)
        cv2.putText(face1, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face1)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

In [ ]:
cap.release()

## Step 2 - Train Model


In [ ]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    user_id=int(os.path.split(image_path)[-1].split("_")[1])
    Labels.append(user_id)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
#saving our created model in a file
model.save('ModelFiles/model.yml')
print("Model trained sucessefully")


## Step 3 - Run Our Facial Recognition

In [2]:
import cv2
import numpy as np
import os
import threading

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

user_id=0
name = ""

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)
final = cv2.face_LBPHFaceRecognizer.create();
final.read('ModelFiles/model.yml')

def show(name,msg):
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        image, face = face_detector(frame)
        image1 = cv2.flip(image,1)
        cv2.putText(image1, "Hey " +name+ " " +msg, (25, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Recognition', image1 )
        if cv2.waitKey(1) == 13:
            break
    cap.release()
    cv2.destroyAllWindows()
    
while True:
    
    ret, frame = cap.read()
    image, face = face_detector(frame)  
    image = np.array(image)
    face = np.array(face)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results = final.predict(face)
        user_id = results[0]
        score = results[1]

        if score < 500:
            confidence = int( 100 * (1 - (score)/400) )
            display_string = str(confidence) + '% Confident it is User'

        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)

        if user_id == 1:
            name = "Gagandeep "
            msg = "Check mail & Whatsapp"
            disp = threading.Thread(target=show, args=(name,msg,))
            disp.start()
            %run ./email_whats.ipynb
            break

        elif user_id == 2:
            name = "Virat"
            msg = "Go, Check AWS"
            disp = threading.Thread(target=show, args=(name,msg,))
            disp.start()
            a = os.system("aws ec2 run-instances  --image-id  ami-0ad704c126371a549  --count 1  --instance-type t2.micro --key-name terraform_key_pair  --subnet-id subnet-a7662aeb   --tag-specifications  ResourceType=instance,Tags=[{Key=Name,Value=ml_launched}]")
            if a == 0:
                print("\t\t\t Success : Instance Launched")
            b = os.system("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone ap-south-1b  --tag-specifications  ResourceType=volume,Tags=[{Key=Name,Value=ml_vol}]")
            if b == 0:
                print("\t\t\t Success : Volume Created")
            import subprocess
            import time
            time.sleep(50)
            ins_id = subprocess.getoutput("aws ec2 describe-instances --filters Name=tag:Name,Values=ml_launched --query Reservations[*].Instances[*].[InstanceId] --output text")
            vol_id = subprocess.getoutput("aws ec2 describe-volumes  --filters Name=tag:Name,Values=ml_vol* --query Volumes[*].[VolumeId] --output text")
            c = os.system("aws ec2 attach-volume --instance-id {} --volume-id {} --device /dev/sdf".format(ins_id,vol_id))
            if c == 0:
                print("\t\t\t Success : Volume ATTACHED")
            break
            
        else:
            name = "Unknown"
            disp = threading.Thread(target=show, args=(name,msg,))
            disp.start()
            
    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

			 Success : Instance Launched
			 Success : Volume Created
			 Success : Volume ATTACHED


## Thank You Task Successfully Completed

In [3]:
ins_id

'i-0e078b27f4a3598fe'

In [4]:
vol_id

'vol-05c1185cec421e97e'